In [3]:
sys.path.append("..")
import os
import utils
import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

### Read data from Norway

In [2]:
data_daily = pd.read_csv("../data/fhi_data_daily.csv")
# data_daily = data_daily[data_daily.year.astype(int) < 2021].reset_index() uncomment if only plotting data fro 2021 
data_daily = data_daily[['date', 'I_new', 'I_cumulative', 'D_new', 'D_cumulative']]
data_daily.date = data_daily['date'].apply(lambda x: 
                                        x.split("/")[1].zfill(2) + "/"
                                        + x.split("/")[0].zfill(2) + "/"
                                        + x.split("/")[2].zfill(2))
data_daily.head()

,date,I_new,I_cumulative,D_new,D_cumulative
0,21/02/2020,1.0,1.0,0.0,0.0
1,22/02/2020,0.0,1.0,0.0,0.0
2,23/02/2020,0.0,1.0,0.0,0.0
3,24/02/2020,0.0,1.0,0.0,0.0
4,25/02/2020,0.0,1.0,0.0,0.0


### Read output from simulations

In [10]:
dir_path = "../"
relative_path = "results/500_simulations_no_vaccines_2021_05_30_13_13_03"
path = dir_path + relative_path
mdp_paths_path   =   f"{path}/mdp_paths.pkl"
start_date_population_age_labels_path = f"{path}/start_date_population_age_labels.pkl"

paths = utils.create_named_tuple('paths', 'filepaths.txt')
config = utils.create_named_tuple('config', paths.config)

start_date, population, age_labels = utils.read_pickle(start_date_population_age_labels_path)
mdp_paths = utils.read_pickle(mdp_paths_path)
mdp_path = mdp_paths[-1]

# Retrive data
dates = []
num_simulations = len(mdp_paths)
num_simulation_weeks = len(mdp_paths[0])

vaccinated_per_age_group =  [[0] * num_simulation_weeks] * num_simulations
vaccinated_per_region =  [[0] * num_simulation_weeks] * num_simulations
final_deaths = np.zeros((num_simulations, num_simulation_weeks))
final_infected = np.zeros((num_simulations, num_simulation_weeks))

for i in range(num_simulations):
    for j in range(num_simulation_weeks):
        if i == 0:
            dates.append(mdp_paths[i][j].date)
        final_deaths[i][j] = np.sum(mdp_paths[i][j].new_deaths)
        final_infected[i][j] = np.sum(mdp_paths[i][j].new_infected)
        if j < num_simulation_weeks -1:
            vaccinated_per_age_group[i][j] = np.sum(mdp_paths[i][j].V, axis=0, dtype=int) - np.sum(mdp_paths[i-1][j-1].V, axis=0, dtype=int) 
            vaccinated_per_region[i][j] = np.sum(mdp_paths[i][j].V, axis=1, dtype=int) - np.sum(mdp_paths[i-1][j-1].V, axis=1, dtype=int) 

### Vaccination

In [ ]:
vaccinated_per_region[6][60]

In [ ]:
vaccinated_per_age_group[6][60]

### Find cumulative cases of deaths and infected from model

In [ ]:
percentile_limit=90

deaths_cumulative = final_deaths.cumsum(axis=1)
deaths_mean = final_deaths.mean(axis=0)
deaths_mean_cumulative = deaths_cumulative.mean(axis=0)
u_percentile_deaths = np.percentile(final_deaths, percentile_limit, axis=0)
l_percentile_deaths = np.percentile(final_deaths, 100-percentile_limit, axis=0)
u_percentile_deaths_cum = np.percentile(deaths_cumulative, percentile_limit, axis=0)
l_percentile_deaths_cum = np.percentile(deaths_cumulative, 100-percentile_limit, axis=0)

infected_cumulative = final_infected.cumsum(axis=1)
infected_mean = final_infected.mean(axis=0)
infected_mean_cumulative = infected_cumulative.mean(axis=0)
u_percentile_infected = np.percentile(final_infected, percentile_limit, axis=0)
l_percentile_infected = np.percentile(final_infected, 100-percentile_limit, axis=0)
u_percentile_infected_cum = np.percentile(infected_cumulative, percentile_limit, axis=0)
l_percentile_infected_cum = np.percentile(infected_cumulative, 100-percentile_limit, axis=0)

In [ ]:
restrict_to_real_data_length = False
max_weeks_real_data = 45

fig, ax = plt.subplots(1)
#ax.plot(dates, deaths_mean, lw=2, label='Deaths model', color='blue')

if restrict_to_real_data_length:
    ax.plot(dates[:max_weeks_real_data], deaths_mean_cumulative[:max_weeks_real_data], lw=2, label='Cumulative fatality cases model', color='orange')
    ax.plot(dates[:max_weeks_real_data], data_daily.D_cumulative.to_numpy(dtype=int)[:max_weeks_real_data*7:7], lw=2, label='Cumulative fatality cases Norway', color='blue')
    #ax.fill_between(dates, u_percentile_deaths, l_percentile_deaths, facecolor='blue', alpha=0.25)
    ax.fill_between(dates[:max_weeks_real_data], u_percentile_deaths_cum[:max_weeks_real_data], l_percentile_deaths_cum[:max_weeks_real_data], facecolor='orange', alpha=0.25)
else:
    ax.plot(dates[:-1], deaths_mean_cumulative[:-1], lw=2, label='Cumulative fatality cases model', color='orange')
    ax.plot(dates[:max_weeks_real_data], data_daily.D_cumulative.to_numpy(dtype=int)[:max_weeks_real_data*7:7], lw=2, label='Cumulative fatality cases Norway', color='blue')
    #ax.fill_between(dates, u_percentile_deaths, l_percentile_deaths, facecolor='blue', alpha=0.25)
    ax.fill_between(dates, u_percentile_deaths_cum, l_percentile_deaths_cum, facecolor='orange', alpha=0.25)
#ax.set_title(r'Cumulative and new deaths per week')
ax.legend(loc='upper left')

# Format the date into months & days
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%U')) 
# Change the tick interval
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=28)) 
# Puts x-axis labels on an angle
plt.gca().xaxis.set_tick_params(rotation = 30)
ax.set_xlabel("Week")

fig.savefig("../plots/computational_study/deaths_comparison.png", bbox_inches='tight', dpi=150)

In [ ]:
restrict_to_real_data_length = True
max_weeks_real_data = 45

fig, ax = plt.subplots(1)
#ax.plot(dates, deaths_mean, lw=2, label='Deaths model', color='blue')
if restrict_to_real_data_length:
    ax.plot(dates[:max_weeks_real_data], infected_mean_cumulative[:max_weeks_real_data], lw=2, label='Cumulative infection cases model', color='orange')
    ax.plot(dates[:max_weeks_real_data], data_daily.I_cumulative.to_numpy(dtype=int)[:max_weeks_real_data*7:7], lw=2, label='Cumulative infection cases Norway', color='blue')
    ax.fill_between(dates[:max_weeks_real_data], u_percentile_infected_cum[:max_weeks_real_data], l_percentile_infected_cum[:max_weeks_real_data], facecolor='orange', alpha=0.25)
else:
    ax.plot(dates[:-1], infected_mean_cumulative[:-1], lw=2, label='Cumulative infection cases model', color='orange')
    ax.plot(dates[:max_weeks_real_data], data_daily.I_cumulative.to_numpy(dtype=int)[:max_weeks_real_data*7:7], lw=2, label='Cumulative infection cases Norway', color='blue')
    #ax.fill_between(dates, u_percentile_deaths, l_percentile_deaths, facecolor='blue', alpha=0.25)
    ax.fill_between(dates, u_percentile_infected_cum, l_percentile_infected_cum, facecolor='orange', alpha=0.25)

#ax.set_title(r'Cumulative and new infected per week')
ax.legend(loc='upper left')

# Format the date into months & days
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%U')) 
# Change the tick interval
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=28)) 
# Puts x-axis labels on an angle
plt.gca().xaxis.set_tick_params(rotation = 30) 
ax.set_xlabel("Week")

fig.savefig("../plots/computational_study/infected_comparison.png", bbox_inches='tight', dpi=150)